In [1]:
import json
from pprint import pp

In [2]:
import sys
sys.path.append('..')

from process_dictionary import group_senses, parse_multiple_form_of

def exclude_keys(dict: dict, keys: list):
  return {k: v for k, v in dict.items() if k not in keys}

In [3]:
# all_entries = []
# # with open("../../../../../Downloads/kaikki.org-dictionary-Spanish.jsonl") as file:
# with open("../../../wiktionary-dictionaries/spanish_dict.json") as file:
#     for line in file:
#         all_entries.append(exclude_keys(json.loads(line), ["forms"]) )

In [4]:
# problematic_groups = []
# less_problematic_groups = []
# for entry in all_entries:
#     sense_groups = group_senses(entry)
#     for group in sense_groups:
#         if group[0].get("form_of") and len(group) != 1 and not any([len(sense['glosses']) != 2 for sense in group]):
#             if not group[0].get("raw_glosses"):
#                 problematic_groups.append((entry, group))
#             else:
#                 less_problematic_groups.append((entry, group, parse_multiple_form_of(group)))

In [5]:
def remove_ending_colon(s):
  if s.endswith(":"):
    return s[:-1]
  return s

def parse_multiple_form_of_raw_glosses(group):
  # in 99.86% of cases for spanish we're dealing with an nice nested inflected form intersection, which all have `glosses` len 2. for the rest we do this:
  if any([len(sense['glosses']) != 2 for sense in group]):
    return [{"definition": sense['glosses'][0]} for sense in group]
  else:
    # lemma_string = remove_ending_colon(group[0]['raw_glosses'][0].split(" of ")[1].strip())
    lemma_string = remove_ending_colon(group[0]['glosses'][0].split("\n")[0].split(" of ")[1].strip())
    result = []
    for sense in group:
      definition = sense['glosses'][-1]

      if not definition.endswith(lemma_string):
        definition += " of " + lemma_string

      result.append({"definition": definition})
    return result

def parse_multiple_form_of_plain_glosses(group):
  # in 99.86% of cases for spanish we're dealing with an nice nested inflected form intersection, which all have `glosses` len 2. for the rest we do this:
  if any([len(sense['glosses']) != 2 for sense in group]):
    return [{"definition": sense['glosses'][0]} for sense in group]
  else:
    lemma_string = remove_ending_colon(group[0]['raw_glosses'][0].split(" of ")[1].strip())
    # lemma_string = remove_ending_colon(group[0]['glosses'][0].split("\n")[0].split(" of ")[1].strip())
    result = []
    for sense in group:
      definition = sense['glosses'][-1]

      if not definition.endswith(lemma_string):
        definition += " of " + lemma_string

      result.append({"definition": definition})
    return result

In [6]:
from collections import defaultdict

In [7]:
multiple_form_of_definitions_raw_glosses = defaultdict(set)
multiple_form_of_definitions_plain_glosses = defaultdict(set)

# note this is the kaikki en-spanish from 2024-05 ish
with open("../../../wiktionary-dictionaries/spanish_dict.json") as file:
    for line in file:
        entry = json.loads(line)
        word = entry.get("word")
        sense_groups = group_senses(entry)
        for group in sense_groups:
            if group[0].get("form_of") and len(group) != 1 and not any([len(sense['glosses']) != 2 for sense in group]):
                for definition in parse_multiple_form_of_raw_glosses(group):
                    multiple_form_of_definitions_raw_glosses[word].add(definition['definition'])
                for definition in parse_multiple_form_of_plain_glosses(group):
                    multiple_form_of_definitions_plain_glosses[word].add(definition['definition'])
                # multiple_form_of_definitions_raw_glosses[word].add(tuple())
                # multiple_form_of_definitions_plain_glosses[word].add(tuple(parse_multiple_form_of_plain_glosses(group)))

In [13]:
[{"word": word, "raw_set": raw_values, "plain_set": plain_values} for word, raw_values, plain_values in zip(multiple_form_of_definitions_raw_glosses.keys(), multiple_form_of_definitions_raw_glosses.values(), multiple_form_of_definitions_plain_glosses.values()) if raw_values != plain_values]


[]

In [12]:
# print(len(problematic_groups), len(less_problematic_groups))

In [11]:
# less_problematic_groups[-1][0]
# less_problematic_groups[-1]

In [10]:
# pp(problematic_groups[::1000][-1][0]['senses'])
# pp(problematic_groups[::1000][-1][1])